**Metodología**
1. Cargar configuración YAML (Clean Code + SOLID).
2. Ejecutar pipeline modular (loader, transformer, writer).
3. Consumir tablas limpias desde `data/processed` para visualizar insights.

In [5]:
from __future__ import annotations

import sys
from pathlib import Path

import pandas as pd
import plotly.express as px

PROJECT_ROOT = Path.cwd()
if not (PROJECT_ROOT / 'config').exists():
    PROJECT_ROOT = PROJECT_ROOT.parent
SRC_PATH = PROJECT_ROOT / 'src'
if str(SRC_PATH) not in sys.path:
    sys.path.insert(0, str(SRC_PATH))

from etl import ETLConfig, ETLPipeline
from etl.io_handlers import CSVDataSource, CSVDataWriter
from etl.transformer import FondequipTransformer

config = ETLConfig.from_yaml(PROJECT_ROOT / 'config/base.yaml')
loader = CSVDataSource(
    path=PROJECT_ROOT / config.dataset.raw_path,
    delimiter=config.dataset.delimiter,
    encoding=config.dataset.encoding,
)
transformer = FondequipTransformer(config.processing)
writer = CSVDataWriter()
pipeline = ETLPipeline(loader, transformer, writer, config.outputs)
result_paths = pipeline.run()
result_paths

ETLResult(clean_table=PosixPath('data/processed/fondequip_clean.csv'), summary_by_region=PosixPath('data/processed/fondequip_resumen_region.csv'), summary_by_type=PosixPath('data/processed/fondequip_resumen_tipo.csv'), summary_by_year=PosixPath('data/processed/fondequip_resumen_anio.csv'))

In [6]:
clean_df = pd.read_csv(PROJECT_ROOT / config.outputs.clean_table)
clean_df.head(3)

,folio,programa,institucion,region_instalacion,tipo_equipamiento,equipo,marca,modelo,coordinador,email,anio_convocatoria
0,EQY240004,FONDEQUIP,UNIVERSIDAD SAN SEBASTIAN,Región de Los Ríos,Otros,LASER CONFOCAL SCANNING MICROSCOPE,Leica,Stellaris 8,Luis Barros,fbarros@cecs.cl,2024
1,EQY230023,FONDEQUIP,UNIVERSIDAD AUSTRAL DE CHILE,Región de Los Ríos,Otros,NEXTSEQ 2000 SYSTEM,Illumina,20038897,Andrea Silva,andrea.silva@uach.cl,2023
2,EQY230021,FONDEQUIP,UNIVERSIDAD DE CONCEPCION,Región del Bío-Bío,Otros,MGI GENETIC INTEGRATED PLATFORM,MGI,MGI MGISP-100RS MGI Genetic Sequencer DNBSEQ-G...,Felipe Zuñiga,fzuniga@udec.cl,2023


In [8]:
import nbformat, sys
nbformat.__version__, sys.executable

('5.10.4',
 '/Users/brunosanmartinnavarro/Documents/UACh/anid-fondequip-equipment/.venv/bin/python')

In [12]:
import nbformat
import plotly.io._renderers as _renderers
_renderers.nbformat = nbformat
nbformat.__version__

'5.10.4'

## Vista ejecutiva

In [16]:
region_df = pd.read_csv(PROJECT_ROOT / config.outputs.summary_by_region)
fig = px.bar(
    region_df,
    x='region_instalacion',
    y='total_equipos',
    text_auto='.0f',
    title='Equipos por región de instalación',
    labels={'region_instalacion': 'Región', 'total_equipos': 'Equipos'}
)
fig.update_layout(xaxis_tickangle=-30)
fig.show()
region_df.head()

,region_instalacion,total_equipos
0,Región del Bío-Bío,59
1,Región de Los Ríos,29
2,Región de la Araucanía,21
3,Región de los Lagos,5


In [14]:
tipo_df = pd.read_csv(PROJECT_ROOT / config.outputs.summary_by_type)
fig = px.bar(
    tipo_df,
    x='tipo_equipamiento',
    y='total_equipos',
    text_auto='.0f',
    title='Distribución por tipo de equipamiento',
    labels={'tipo_equipamiento': 'Tipo', 'total_equipos': 'Equipos'}
)
fig.update_layout(xaxis_tickangle=-20)
fig.show()
tipo_df.head()

,tipo_equipamiento,total_equipos
0,Otros,49
1,Cromatógrafos y Espectrómetros,18
2,Instrumentos Bioanalíticos,17
3,Equipos de Procesamiento y ensayo de Materiales,14
4,Microscopios y Difractómetros,14


In [15]:
anio_df = pd.read_csv(PROJECT_ROOT / config.outputs.summary_by_year)
fig = px.line(
    anio_df,
    x='anio_convocatoria',
    y='total_equipos',
    markers=True,
    title='Evolución anual de adjudicaciones',
    labels={'anio_convocatoria': 'Año', 'total_equipos': 'Equipos'}
)
fig.show()
anio_df

,anio_convocatoria,total_equipos
0,2013,10
1,2014,12
2,2015,23
3,2016,8
4,2017,7
5,2018,12
6,2019,3
7,2020,6
8,2021,9
9,2022,7
